In [352]:
import numpy as np
from qiskit import *
from random import random, randint
import matplotlib.pyplot as plt
from qiskit.circuit.library import ZZFeatureMap, RealAmplitudes
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
from qiskit import QuantumCircuit

In [353]:
X_train=open('0val_train')
X_train=X_train.readlines()
X_train_new=[]
for i in X_train:
    if i != '\n':
        X_train_new.append(i)
for i in range(len(X_train_new)):
    X_train_new[i]=X_train_new[i].replace('\n',"")
    X_train_new[i]=X_train_new[i].split(',')
for i in range(len(X_train_new)):
    for j in range(len(X_train_new[i])):
            X_train_new[i][j]=float(X_train_new[i][j])
X_train=X_train_new
X_test=open('0val_test')
X_test=X_test.readlines()
X_test_new=[]
for i in X_test:
    if i != '\n':
        X_test_new.append(i)
for i in range(len(X_test_new)):
    X_test_new[i]=X_test_new[i].replace('\n',"")
    X_test_new[i]=X_test_new[i].split(',')
for i in range(len(X_test_new)):
    for j in range(len(X_test_new[i])):
            X_test_new[i][j]=float(X_test_new[i][j])
X_test=X_test_new
Y_train=open('0check_train')
Y_train=Y_train.readlines()
Y_train=Y_train[0].split(',')
for i in range(len(Y_train)):
     Y_train[i]=int(Y_train[i])

Y_test=open('0check_test')
Y_test=Y_test.readlines()
Y_test=Y_test[0].split(',')
for i in range(len(Y_test)):
     Y_test[i]=int(Y_test[i])
Y_train.extend(Y_test)
X_train.extend(X_test)

In [354]:
def create_circuit(feature_data):
    feature_data = np.reshape(preprocessing.normalize([feature_data]), len(feature_data))
    circuit = ZZFeatureMap(2)
    circuit.compose(RealAmplitudes(2))
    circuit.initialize(feature_data, circuit.qubits)
    return circuit

def parity_probabilities(counts, shots=100):
    class_0 = 0
    class_1 = 0
    for keyBefore, value in counts.items():
        key = keyBefore.split()[0]
        temp = int(key[0]) ^ int(key[1])
        for i in range(2, len(key)):
            temp = temp ^ int(key[i])
        if temp == 0:
            class_0 += value
        else:
            class_1 += value
    return {0: class_0 / shots, 1: class_1 / shots}

def process_output(circuit, s):
    arr = []
    simulator = Aer.get_backend('aer_simulator')
    job = execute(circuit, simulator, shots=s)
    result = job.result()
    counts = result.get_counts(circuit)
    return counts

In [355]:
def mse(num_shots, expected_value):
    probability_of_expected = num_shots[expected_value]
    return (1 - probability_of_expected) ** 2

def count_costs(x_values, y_values, angles, received_measurements):
    cost = 0
    for i in range(len(x_values)):
        cost += mse(received_measurements[i], y_values[i])
    return cost / len(x_values)


In [356]:
def cost(num_qubits, x, y, angles):
    measured = []
    for i in x:
        qc = create_circuit(i)
        measured.append(measure(qc))
    cost_count = cost(x, y, angles, measured)
    return cost_count

In [357]:
def train(x, y):
    initial_angles = [2 * np.pi * random() for i in range(2)]
    optimized = minimize(lambda angles: cost(2, x, y, angles), initial_angles, method='COBYLA')
    return optimized


In [358]:
final_obj = train(X_train, Y_train)

AerError: 'circuits have parameters but parameter_binds is not specified.'

In [ ]:
final_array = final_obj.x.tolist()
def test_accuracy(x, y, optimized_params):
    measured = []
    num_qubits = 2
    for i in range(len(x)):
        qc = create_circuit(x[i])
        measured.append(measure(qc))
    actual_measurements = []
    for i in measured:
        if i[0] > i[1]:
            actual_measurements.append(0)
        else:
            actual_measurements.append(1)
    count_correct = 0
    for i in range(len(y)):
        if y[i] == actual_measurements[i]:
            count_correct += 1
    return count_correct / len(y)

print("Accuracy:", test_accuracy(features_train, target_train, final_array))

Accuracy: 0.375
